<ol>
    <h1>Упражнение 1 - Генератор случайных матриц</h1>
    Реализовать генератор матриц, который должен поддерживать функции:
    <li>Генерация абсолютно случайной матрицы $ n \times m $;</li>
    <li>Генерация случайной диагональной матрицы $ n \times n $;</li>
    <li>Генерация случайной верхнетреугольной матрицы;</li>
    <li>Генерация случайной нижнетреугольной матрицы;</li>
    <li>Генерация симметричной матрицы</li>
    <li>Генерация вырожденной матрицы</li>
    <li>Генерация матрицы ступенчатого вида $ n \times n $ ранга $ m $;</li>
    <li>Генерация возмущения матрицы $ n \times m $, каждый элемент которой не превосходит по модулю заданный $ \epsilon $. Оценить величину нормы матрицы возмущений в зависимости от параметра $ \epsilon $ (оценить верхную границу);</li>
    Оценить численно вероятность того, что созданная матрица будет вырожденной для какого-либо случая выше.
</ol>
<hr>

In [35]:
import numpy as np

def matrix_generate(rows, columns, type_ = "full", rank = 0, eps = 0):
    """
    matrix_generate(rows, columns, type_ = "full")

    Создаёт случайную матрицу выбранного типа.

    Если матрицу нужных размеров создать нельзя должен выдать
    строку f"Error with type {type_} and shape ({rows},{columns})".

    Parameters
    ----------

    rows : int
        Количество строк в создаваемой матрице.
    columns : int
        Количество столбцов в создаваемой матрице.
    type_ : str, optional
        Тип создаваемой матрицы: "full", "upper_triangular", "symmetric" и т.д.
    rank : int, optional
        Ранг создаваемой матрицы, используется при генерации некоторых типов матриц
    eps: float, optional
        Дополнительное число, использующееся при генерации для некоторых типов матриц.

    Returns
    -------
    out : ndarray or str
        Выдаёт матрицу нужного типа либо ошибку.

    Notes
    -----
    Поддерживаемые типы матриц:
        "full" - абсолютно случайная матрица
        "diagonal" - случайная диагональная матрица
        "upper_triangular", "lower_triangular" - случайные верхне- и нижне-треугольные матрицы
        "symmetric" - случайная симметричная матрица
        "singular" - случайная вырожденная матрица
        "perturbation" - матрица возмущения
    """
    A = None
    if type_ == "full":
        A = np.random.random(size=(rows, columns))
    elif type_ == "diagonal":
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        else:
            A = np.diag(np.diag(np.random.random(size=(rows, columns))))
    elif type_ == "upper_triangular":
        A = np.random.random(size=(rows, columns))
        A = np.triu(A)
    elif type_ == "lower_triangular":
        A = np.random.random(size=(rows, columns))
        A = np.tril(A)
    elif type_ == "symmetric":
    # Идея очень проста: создадим матрицу из верхнетреугольной и нижнетреугольной транспортированной, затем вычтем одну диагональную часть,
    # так как учли её дважды
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        else:
            A = np.random.random(size=(rows, columns))
            A = np.tril(A.T) + np.triu(A) - np.diag(np.diag(A))
    elif type_ == "singular":
    # Здесь воспользуемся такой идеей: ортогональная матрица обратима и не меняет определитель и ранг при произведении,
    # следовательно создадим точно необратимую матрицу (например, диагональную с нулями и единицами на диагонали),
    # затем умножим её на случайную ортогональную
        if rows != columns:
            return f"Error with type {type_} and shape ({rows},{columns})"
        else:
            I = np.ones(rank)
            for i in range(rows - rank):
                I = np.insert(I, np.random.randint(len(I)), 0)
            S = np.diag(I)
            U, R = np.linalg.qr(np.random.random(size=(rows, columns)))
            V, R = np.linalg.qr(np.random.random(size=(rows, columns)))
            A = U @ S @ V.T

    elif type_ == "perturbation":
        if eps < 0:
            return f"Error with epsilon value {eps}"
        else:
            A = np.random.uniform(-eps, eps, size=(rows, columns))
    else:
        return f"Error, no type {type_}"
    return A

In [2]:
matrix_generate(3, 4, "full")

array([[0.62370401, 0.70209783, 0.65718191, 0.10026853],
       [0.89059901, 0.19173117, 0.99872878, 0.58070429],
       [0.40923407, 0.87239488, 0.56842024, 0.95699815]])

In [3]:
matrix_generate(4, 4, "diagonal")

array([[0.04224581, 0.        , 0.        , 0.        ],
       [0.        , 0.21249988, 0.        , 0.        ],
       [0.        , 0.        , 0.69479468, 0.        ],
       [0.        , 0.        , 0.        , 0.3585665 ]])

In [4]:
matrix_generate(4, 3, "diagonal")

'Error with type diagonal and shape (4,3)'

In [5]:
matrix_generate(5, 3, "upper_triangular")

array([[0.34010191, 0.66805809, 0.13682258],
       [0.        , 0.88638867, 0.31287517],
       [0.        , 0.        , 0.73855819],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [6]:
matrix_generate(3, 5, "upper_triangular")

array([[0.98530998, 0.33170705, 0.70589054, 0.85259422, 0.4719688 ],
       [0.        , 0.59549291, 0.34242651, 0.87184709, 0.27077522],
       [0.        , 0.        , 0.04300856, 0.1390998 , 0.07167572]])

In [9]:
matrix_generate(5, 3, "lower_triangular")

array([[0.79132855, 0.        , 0.        ],
       [0.45430929, 0.50898886, 0.        ],
       [0.30597536, 0.02141311, 0.76352188],
       [0.41446375, 0.14371691, 0.00581857],
       [0.06942215, 0.0593483 , 0.39897003]])

In [10]:
matrix_generate(2, 5, "lower_triangular")

array([[0.45498883, 0.        , 0.        , 0.        , 0.        ],
       [0.82059551, 0.15923194, 0.        , 0.        , 0.        ]])

In [14]:
matrix_generate(4, 4, "symmetric")

array([[0.22286764, 0.50962892, 0.3704866 , 0.87214974],
       [0.50962892, 0.10850153, 0.80639691, 0.24036443],
       [0.3704866 , 0.80639691, 0.83783059, 0.2083882 ],
       [0.87214974, 0.24036443, 0.2083882 , 0.81587267]])

In [15]:
matrix_generate(5, 3, "symmetric")

'Error with type symmetric and shape (5,3)'

In [24]:
matrix_generate(5, 3, "singular", 3)

'Error with type singular and shape (5,3)'

In [31]:
matrix_generate(4, 4, "singular", 3)

array([[ 0.16342882, -0.2792828 ,  0.29672842,  0.65841474],
       [ 0.58184064,  0.45230312, -0.51767427, -0.10973766],
       [ 0.2108097 ,  0.6276562 ,  0.74406057, -0.05976095],
       [ 0.49558831, -0.03328401, -0.12848463,  0.54016141]])

In [43]:
matrix_generate(3, 5, "perturbation", eps = 0.02)

array([[-0.01755228,  0.00149051, -0.00519523,  0.0188135 , -0.00459226],
       [ 0.00943887, -0.00875644, -0.01208807, -0.00988858, -0.00543055],
       [ 0.01496368, -0.01284969,  0.01350437,  0.01100133, -0.00840227]])